## KFF data handling

This Notebook will handle all the www.kff.org data.
The following notebook consist all the functions needed for handling the expored datasets from the KFF.org website.

Each Dataset is treated individually in a seperated block via the following functions.
The exported datasets consist various data on a state level. For example:
* Economicle state of diffrent population by ethnicity groups
* GSP of the state
* Poverty of diffrent age groups 
* Democrat/ Republican 

Each dataset is edited and merged into a single dataframe called *df_Y_S_KFF* which eventually hold all the exported data and  the crime statistics of each State in USA.

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # spread dataframe columns 

In [2]:
#-----------New folder named bar
newpath = r'KFF' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [3]:
def concat_suicide(df):
    dfs_list = []

    for idx,i in enumerate(df.columns):
        if (i!='State'):
            temp = df[['State',i]].copy()
            temp['Year']= i[:4]
            temp.rename(columns={temp.columns[1]:i[6:]}, inplace=True)
            dfs_list.append(temp)

    concat_suicide = pd.concat(dfs_list,axis=0) 
    concat_suicide = concat_suicide[[concat_suicide.columns[0],concat_suicide.columns[2],concat_suicide.columns[1]]]
    concat_suicide = concat_suicide.astype({"Year": int})
    dfs_list
    return concat_suicide

In [4]:
def concat_political_state(df):
    
    dfs_list = []
    dfcopy=df.copy()
    for i in range(10):
        if (dfcopy.columns[1]!='State'):
            temp = dfcopy[['State',dfcopy.columns[1]]].copy()
            temp['Year']= 2010+i
            #temp.rename(columns={temp.columns[1]:i[6:]}, inplace=True)
            dfs_list.append(temp)

    concat_suicide = pd.concat(dfs_list,axis=0) 
    concat_suicide = concat_suicide[[concat_suicide.columns[0],concat_suicide.columns[2],concat_suicide.columns[1]]]
    concat_suicide = concat_suicide.astype({"Year": int})
    dfs_list
    return concat_suicide


In [5]:
def concat_suicide_multy_year(df,numofmulty):
    
    dfs_list = []
    dfcopy=df.copy()
    for j in range(int((len(dfcopy.columns)-1)/numofmulty)): 
        begin=int((j*numofmulty)+1)
        end=int((j*numofmulty)+1+numofmulty)
        temp2=dfcopy.iloc[:,begin:end]
        for idx,k in enumerate(temp2.columns):
            year=k[:4]
            temp2.rename(columns={temp2.columns[idx]:k[6:]}, inplace=True)
        od_to_concat=dfcopy.iloc[:,0:1]#states
        
        od_to_concat.insert(1, 'Year', year)
        od_to_concat = pd.concat([od_to_concat, temp2], axis=1)
        dfs_list.append(od_to_concat)

    concat_suicide = pd.concat(dfs_list,axis=0) 
    concat_suicide = concat_suicide.astype({"Year": int})
    return concat_suicide


In [6]:
# Editing State column 
def fit_State_val_column(df):
    df.rename(columns={"Location": "State"}, inplace=True)
    df.sort_values(by=['State'],inplace=True)
    df['State'] = df['State'].str.upper()
    return df


In [7]:
## merge the data set into main df 
def marge_general_with_crimes(df_Y_S,dfKFF):
    result = pd.merge(df_Y_S, dfKFF, how="outer", on=['State', 'Year'])
    return result

In [8]:
## Addin missing years 

def edding_year_missing_values(df,year,place, method, name=None, select_rows=None):
    
    #method: 'max+std', 'min-std' , 'mean+std'
    
    location = df['Location'] #save for later
    df.drop(columns='Location', inplace=True) #make dtype equal
    
    if method == 'min-std':
        std_col=df.transpose().std()
        min_col=df.transpose().min()
        x=min_col-std_col #create dissairble column
    elif method == 'mean+std':
        if select_rows == None:
            std_col=df.transpose().std()
            mean_col=df.transpose().mean()
            x=mean_col+std_col #create dissairble column
        else:
            std_col=df.transpose().iloc[select_rows[0]::select_rows[1], :].std()
            mean_col=df.transpose().iloc[select_rows[0]::select_rows[1], :].mean()
            x=mean_col+std_col #create dissairble column
    elif method == 'max+std':
        if select_rows == None:
            std_col=df.transpose().std()
            max_col=df.transpose().max()
            x=max_col+std_col #create dissairble column
        else:
            std_col=df.transpose().iloc[select_rows[0]::select_rows[1], :].std()
            max_col=df.transpose().iloc[select_rows[0]::select_rows[1], :].max()
            x=max_col+std_col #create dissairble column
             
        
    if name==None:
        col_name=df.columns[0]
        col_name = str(year)+col_name[4:]
    else:
        col_name=name
    
    df.insert(place,col_name,value=x.astype(int)) #place=where to insert the column
    df.insert(0,'Location',value=location) 
    
    return df


In [9]:
# Search nan and NSD values 
def search_NSD(df):
    for col in df.columns:
        if 'NSD' in df[col].values:
            print('Element exists in col:',str(col))
    print("How many NaN vals:",df.isnull().sum().sum())

In [10]:
## Begin reading files 

path='./proj_datasets/'
name='merge_T_8_and_78.xlsx'
df=pd.read_excel(path+name, index_col=0) # read each file in the folder
df_Y_S=df.groupby(['Year','State']).sum().reset_index()
dflist = []
direct='./proj_datasets/KFF_2/'

In [11]:
#Adding total police force and population ratio :
cop = df_Y_S['Total_Police force']
pop = df_Y_S['Population']
df_Y_S['Police Force Population Ratio'] = cop / pop

In [13]:
## filling Nan values with mode

def filling_NaN_NSD(df, sum_total = False):
    
    ## Summtion  to Total column  + filling nan
    if sum_total == True:
        df_cols_list = list(df.columns[2:-1]) # getting col list
        df[df_cols_list] = df[df_cols_list].apply(pd.to_numeric, errors='coerce') ## to numeric 
        for  col in (df_cols_list):
            df[col].fillna(value = df[col].mode()[0],inplace = True) ## filling each cell with mode
        df[df.columns[-1]] = df[df_cols_list].sum(axis = 1)  # sum each raw
    ## filing nan
    else:
        df_cols_list = list(df.columns[2:]) # getting col list
        df[df_cols_list] = df[df_cols_list].apply(pd.to_numeric, errors='coerce') ## to numeric 
        for  col in (df_cols_list):
            df[col].fillna(value = df[col].mode()[0],inplace = True) ## filling each cell with mode

    return df

In [12]:
path=direct+'Suicide_Rate.csv'
dfSuicide = pd.read_csv(path,skiprows=2)
dfSuicide.dropna(inplace=True)
dfSuicide=fit_State_val_column(dfSuicide)
dfSuicide=concat_suicide(dfSuicide)
dfSuicide=marge_general_with_crimes(df_Y_S,dfSuicide)
dfSuicide.dropna(thresh=10, inplace=True)
dfSuicide.rename(columns={"Suicide Rate per 100,000 Individuals": "Suicide Rate P100K"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S, dfSuicide, how="outer")


In [14]:
path=direct+'Drug Overdose Death Rate.csv'#per 100,000
df_OD = pd.read_csv(path,skiprows=2)
df_OD.dropna(inplace=True)
df_OD=fit_State_val_column(df_OD)
df_OD=concat_suicide(df_OD)
df_OD=filling_NaN_NSD(df_OD)
df_OD=marge_general_with_crimes(df_Y_S,df_OD)
df_OD.dropna(thresh=10, inplace=True)
df_OD.rename(columns={"Drug Overdose Deaths": "OD D P100K"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_OD, how="outer")
df_Y_S_KFF.isnull().sum().sum() # checking if there are nan values
df_Y_S_KFF.info()

df_Y_S_KFF

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 487
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Year                                  488 non-null    int64  
 1   State                                 488 non-null    object 
 2   Population                            488 non-null    float64
 3   Violent crime                         488 non-null    float64
 4   Murder and nonnegligent manslaughter  488 non-null    float64
 5   Forcible rape                         488 non-null    float64
 6   Robbery                               488 non-null    float64
 7   Aggravated assault                    488 non-null    float64
 8   Property crime                        488 non-null    float64
 9   Burglary                              488 non-null    float64
 10  Larceny-theft                         488 non-null    float64
 11  Motor vehicle theft

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1


In [15]:
path=direct+'Opioid Overdose Deaths by Type of Opioid.csv'
df_TypedrugOD = pd.read_csv(path,skiprows=2)
df_TypedrugOD.dropna(thresh=20,inplace=True)
df_TypedrugOD.dropna(axis=1,thresh=20,inplace=True)
df_TypedrugOD=fit_State_val_column(df_TypedrugOD)
df_TypedrugOD=concat_suicide_multy_year(df_TypedrugOD, 4)
df_TypedrugOD= filling_NaN_NSD(df_TypedrugOD)

# ## merge 
df_TypedrugOD=marge_general_with_crimes(df_Y_S,df_TypedrugOD)
df_TypedrugOD.dropna(thresh=10, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_TypedrugOD, how="outer")
df_Y_S_KFF


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0


In [16]:
path=direct+'Opioid Overdose Deaths by Race Ethnicity.csv'
df_od_race = pd.read_csv(path,skiprows=2)
df_od_race.dropna(thresh=20,inplace=True)
df_od_race.dropna(axis=1,thresh=20,inplace=True)
df_od_race=fit_State_val_column(df_od_race)
df_od_race=concat_suicide_multy_year(df_od_race, 5)
df_od_race.drop(columns=["Total"],axis=1,inplace = True)  ### droping Other and Total Cols
df_od_race= filling_NaN_NSD(df_od_race)
df_od_race=marge_general_with_crimes(df_Y_S,df_od_race)
df_od_race.dropna(thresh=10, inplace=True)
df_od_race.rename(columns={"White, Non-Hispanic":"Opi ODD White N-H",
                           "Black, Non-Hispanic":"Opi ODD Black N-H",
                           "Hispanic":"Opi ODD Hispanic",
                           "Other":"Opi ODD other",
                           "Total":"Opi ODD Total" }, inplace=True)

df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_od_race, how="outer")
#df_Y_S_KFF.isnull().sum().sum() # checking if there are nan values
df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0


In [17]:
path=direct+'Opioid Overdose Deaths and Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths.csv'
df_od = pd.read_csv(path,skiprows=2)
df_od.dropna(thresh=20,inplace=True)
df_od.dropna(axis=1,thresh=20,inplace=True)
df_od=fit_State_val_column(df_od)
df_od=concat_suicide_multy_year(df_od, 3)
df_od = filling_NaN_NSD(df_od)
df_od=marge_general_with_crimes(df_Y_S,df_od)
df_od.dropna(thresh=10, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_od, how="outer")
df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77


In [ ]:
#Opioid Overdose Deaths by Gender
path=direct+'Opioid Overdose Deaths by Gender.csv'
df_od_gender = pd.read_csv(path,skiprows=2)
df_od_gender.dropna(thresh=20,inplace=True)
df_od_gender.dropna(axis=1,thresh=20,inplace=True)
df_od_gender=fit_State_val_column(df_od_gender)
df_od_gender=concat_suicide_multy_year(df_od_gender, 3)
df_od_gender= filling_NaN_NSD(df_od_gender,True)

## merge 
df_od_gender=marge_general_with_crimes(df_Y_S,df_od_gender)
df_od_gender.dropna(thresh=10, inplace=True)
df_od_gender.rename(columns={"Male":"Opi ODD Male","Female":"Opi ODD Female","Total":"Opi ODD Total Gender" }, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_od_gender, how="outer")

df_Y_S_KFF

In [19]:
#Population Distribution by Race/Ethnicity
## Note; Changed from Precent to Numberic 

path=direct+'Population Distribution by RaceEthnicity.csv'
df_pop_dist = pd.read_csv(path,skiprows=2)
df_pop_dist.dropna(thresh=20,inplace=True)
df_pop_dist.drop(columns='Footnotes', inplace=True)
df_pop_dist=fit_State_val_column(df_pop_dist)
df_pop_dist=concat_suicide_multy_year(df_pop_dist, 8)
df_pop_dist.drop(columns=["Total"],axis=1,inplace = True)
df_pop_dist = filling_NaN_NSD(df_pop_dist)
# # merge 
df_pop_dist=marge_general_with_crimes(df_Y_S,df_pop_dist)
df_pop_dist.dropna(thresh=20, inplace=True)

df_pop_dist.rename(columns={"White":"P_D: White",
                            "Black":"P_D: Black",
                            "Hispanic":"P_D: Hispanic",
                            "Asian":"P_D: Asian",
                            "American Indian/Alaska Native":"P_D: Indian/Alaska Nativ" ,
                            "Native Hawaiian/Other Pacific Islander":"P_D: Hawaiian/Pacific Islander Nativ", 
                            "Multiple Races":"P_D: Multiple Races" ,
                            "Total":"P_D: Total" }, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_pop_dist, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77,621.0,307.0,928.0,4602500.0,349200.0,401700.0,169100.0,37400.0,3100.0,114100.0


In [20]:
#Estimates of Homelessness
path=direct+'Estimates of Homelessness.csv'
df_Homeless = pd.read_csv(path,skiprows=2)
df_Homeless.dropna(thresh=20,inplace=True)
df_Homeless.drop(columns='Footnotes', inplace=True)
df_Homeless=fit_State_val_column(df_Homeless)
df_Homeless.drop(columns=['2020__Individuals','2020__People in Families with Children','2020__Total People Experiencing Homelessness'],inplace=True)
df_Homeless=concat_suicide_multy_year(df_Homeless, 3)
df_Homeless=marge_general_with_crimes(df_Y_S,df_Homeless)
df_Homeless.dropna(axis=0,thresh=8, inplace=True)
df_Homeless.rename(columns={"Individuals":"Individuals Homeless",
                            "People in Families with Children":"People in Families with Children Homeless"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Homeless, how="outer")
df_Homeless
df_Y_S_KFF.shape
df_Y_S_KFF
# df_Homeless

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77,621.0,307.0,928.0,4602500.0,349200.0,401700.0,169100.0,37400.0,3100.0,114100.0,2545.0,1993.0,4538.0


In [21]:
#State Political Parties
path=direct+'State Political Parties.csv'
df_Political_state = pd.read_csv(path,skiprows=2)
df_Political_state.drop(columns='Footnotes', inplace=True)
df_Political_state.dropna(inplace=True)
df_Political_state=fit_State_val_column(df_Political_state)
df_Political_state=concat_political_state(df_Political_state)
df_Political_state=marge_general_with_crimes(df_Y_S,df_Political_state)
df_Political_state.dropna(thresh=8, inplace=True)

# renamimmg cols 
df_Political_state['Republican'] = df_Political_state['Governor Political Affiliation']
df_Political_state['Democrat'] = df_Political_state['Governor Political Affiliation']
df_Political_state['Republican'].replace({'Republican':1 ,'Democrat':0}, inplace=True)
df_Political_state['Democrat'].replace({'Republican':0 ,'Democrat':1}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Political_state, how="outer")

df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0,Democrat,0,1
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0,Republican,1,0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77,621.0,307.0,928.0,4602500.0,349200.0,401700.0,169100.0,37400.0,3100.0,114100.0,2545.0,1993.0,4538.0,Democrat,0,1


In [22]:
#Number of Reported Chlamydia Cases by Gender
path=direct+'Number of Reported Chlamydia Cases by Gender.csv'
df_Chlamydia_gender = pd.read_csv(path,skiprows=2)
df_Chlamydia_gender.drop(columns='Footnotes', inplace=True)
df_Chlamydia_gender.dropna(axis=0,thresh=8, inplace=True)

## filing missing years 
df_Chlamydia_gender=edding_year_missing_values(df_Chlamydia_gender,2019,len(df_Chlamydia_gender.columns)-1, 'mean+std','2019__Men',[1,2])
df_Chlamydia_gender=edding_year_missing_values(df_Chlamydia_gender,2019,len(df_Chlamydia_gender.columns)-1, 'mean+std','2019__Women',[2,2])

df_Chlamydia_gender=fit_State_val_column(df_Chlamydia_gender)
df_Chlamydia_gender=concat_suicide_multy_year(df_Chlamydia_gender, 2)
df_Chlamydia_gender=marge_general_with_crimes(df_Y_S,df_Chlamydia_gender)
df_Chlamydia_gender.dropna(thresh=8, inplace=True)
df_Chlamydia_gender.rename(columns={"Men":"Chlamydia Man",
                            "Women":"Chlamydia Women"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Chlamydia_gender, how="outer")


df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1,27104.0,19017.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0,Democrat,0,1,20447.0,14898.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0,Republican,1,0,3540.0,2349.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77,621.0,307.0,928.0,4602500.0,349200.0,401700.0,169100.0,37400.0,3100.0,114100.0,2545.0,1993.0,4538.0,Democrat,0,1,18303.0,12626.0


In [23]:
#Number of Reported Gonorrhea Cases by Gender
path=direct+'Number of Reported Gonorrhea Cases by Gender.csv'
df_Gonorrhea_gender = pd.read_csv(path,skiprows=2)
df_Gonorrhea_gender.drop(columns='Footnotes', inplace=True)
df_Gonorrhea_gender.dropna(axis=0,thresh=8, inplace=True)

## filling missing years 
df_Gonorrhea_gender=edding_year_missing_values(df_Gonorrhea_gender,2019,len(df_Gonorrhea_gender.columns)-1, 'mean+std','2019__Men',[1,2])
df_Gonorrhea_gender=edding_year_missing_values(df_Gonorrhea_gender,2019,len(df_Gonorrhea_gender.columns)-1, 'mean+std','2019__Women',[2,2])


df_Gonorrhea_gender=fit_State_val_column(df_Gonorrhea_gender)
df_Gonorrhea_gender=concat_suicide_multy_year(df_Gonorrhea_gender, 2)
df_Gonorrhea_gender=marge_general_with_crimes(df_Y_S,df_Gonorrhea_gender)
df_Gonorrhea_gender.dropna(thresh=8, inplace=True)
df_Gonorrhea_gender.rename(columns={"Men":"Gonorrhea Man",
                            "Women":"Gonorrhea Women"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Gonorrhea_gender, how="outer")

df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1,27104.0,19017.0,5155.0,6150.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0,Democrat,0,1,20447.0,14898.0,3380.0,6015.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0,Republican,1,0,3540.0,2349.0,555.0,600.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,237.0,660.0,54.0,292.0,712.0,121.0,53.0,42.0,928.0,1201.0,0.77,621.0,307.0,928.0

In [24]:
#Number of Reported Gonorrhea Cases
path=direct+'Number of Reported Gonorrhea Cases.csv'
df_Gonorrhea_Cases = pd.read_csv(path,skiprows=2)
df_Gonorrhea_Cases.drop(columns='Footnotes', inplace=True)
df_Gonorrhea_Cases.dropna(thresh=10, inplace=True)

# filling missing year
df_Gonorrhea_Cases=edding_year_missing_values(df_Gonorrhea_Cases,2019,len(df_Gonorrhea_Cases.columns)-1, 'max+std')

df_Gonorrhea_Cases=fit_State_val_column(df_Gonorrhea_Cases)
df_Gonorrhea_Cases=concat_suicide(df_Gonorrhea_Cases)
df_Gonorrhea_Cases=marge_general_with_crimes(df_Y_S,df_Gonorrhea_Cases)
df_Gonorrhea_Cases.dropna(inplace=True)

df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Gonorrhea_Cases, how="outer")
df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1,27104.0,19017.0,5155.0,6150.0,14928.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0,Democrat,0,1,20447.0,14898.0,3380.0,6015.0,14348.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0,Republican,1,0,3540.0,2349.0,555.0,600.0,1513.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,9751.0,69749.0,9414.0,53944.0,6391.0,332.0,18960,0.004963,14.0,21.1,2

In [25]:
#Number of Reported Syphilis Cases
path=direct+'Number of Reported Syphilis Cases.csv'
df_Syphilis_Cases = pd.read_csv(path,skiprows=2)
df_Syphilis_Cases.drop(columns='Footnotes', inplace=True)
df_Syphilis_Cases.dropna(thresh=10, inplace=True)
df_Syphilis_Cases['2018__Syphilis Cases'] = df_Syphilis_Cases['2018__Syphilis Cases'].str.replace(r',', '').astype(float)#change type in order to use transpose

#filling missing year
df_Syphilis_Cases=edding_year_missing_values(df_Syphilis_Cases,2019,len(df_Syphilis_Cases.columns)-1, 'mean+std')

df_Syphilis_Cases=fit_State_val_column(df_Syphilis_Cases)
df_Syphilis_Cases=concat_suicide(df_Syphilis_Cases)
df_Syphilis_Cases=marge_general_with_crimes(df_Y_S,df_Syphilis_Cases)
df_Syphilis_Cases.dropna(inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Syphilis_Cases, how="outer")

df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0,6115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1,27104.0,19017.0,5155.0,6150.0,14928.0,1610.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.0,47400.0,371100.0,15985.0,5592.0,21577.0,Democrat,0,1,20447.0,14898.0,3380.0,6015.0,14348.0,1580.0
485,2019,WEST VIRGINIA,173712.0,565.0,10.0,83.0,76.0,396.0,3659.0,689.0,2691.0,279.0,27.0,902,0.005193,18.6,52.8,174.0,572.0,13.0,142.0,624.0,40.0,11.0,11.0,673.0,870.0,0.77,449.0,224.0,673.0,1609000.0,55900.0,24800.0,14200.0,3900.0,3100.0,31200.0,1149.0,248.0,1397.0,Republican,1,0,3540.0,2349.0,555.0,600.0,1513.0,143.0
486,2019,WISCONSIN,3820348.0,14435.0,152.0,1736.0,2796.0,

In [26]:
#Distribution of the Nonelderly Uninsured by Race Ethnicity
path=direct+'Distribution of the Nonelderly Uninsured by Race Ethnicity.csv'
df_Uninsured_Race_Ethnicity = pd.read_csv(path,skiprows=2)
df_Uninsured_Race_Ethnicity.drop(columns='Footnotes', inplace=True)
df_Uninsured_Race_Ethnicity.dropna(thresh=20,inplace=True)
df_Uninsured_Race_Ethnicity=fit_State_val_column(df_Uninsured_Race_Ethnicity)
df_Uninsured_Race_Ethnicity=concat_suicide_multy_year(df_Uninsured_Race_Ethnicity, 7)
df_Uninsured_Race_Ethnicity.drop(columns=["Total"],axis=1,inplace = True)
df_Uninsured_Race_Ethnicity = filling_NaN_NSD(df_Uninsured_Race_Ethnicity)


df_Uninsured_Race_Ethnicity=marge_general_with_crimes(df_Y_S,df_Uninsured_Race_Ethnicity)
df_Uninsured_Race_Ethnicity.dropna(axis=0,thresh=10, inplace=True)
df_Uninsured_Race_Ethnicity.rename(columns={"White":"P_D_Uninsured: White",
                                            "Black":"P_D_Uninsured: Black",
                                            "Hispanic":"P_D_Uninsured: Hispanic",
                                            "Asian/Native Hawaiian and Pacific Islander":"P_D_Uninsured: Asian/Hawaiian",
                                            "American Indian/Alaska Native":"P_D_Uninsured: Indian/Alaska Nativ" ,
                                            "Multiple Races":"P_D_Uninsured: Multiple Races"}, inplace=True)

df_Y_S_KFF = pd.merge(df_Y_S_KFF, df_Uninsured_Race_Ethnicity, how="outer")


df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0,6115.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,1543100.0,787100.0,551900.0,13600.0,3900.0,268300.0,3666.0,2117.0,5783.0,Democrat,0,1,27104.0,19017.0,5155.0,6150.0,14928.0,1610.0,268300.0,128000.0,192400.0,37300.0,4000.0,17200.0
484,2019,WASHINGTON,4493249.0,16299.0,126.0,2242.0,4256.0,9675.0,154056.0,23730.0,112571.0,17755.0,601.0,16578,0.003690,15.9,15.8,204.0,337.0,73.0,346.0,640.0,45.0,62.0,80.0,827.0,1259.0,0.66,550.0,277.0,827.0,5013200.0,282700.0,964700.0,662400.0,82400.

In [27]:
#Distribution of the Nonelderly with Employer Coverage by Race Ethnicity
path=direct+"Distribution of the Nonelderly Uninsured by Race Ethnicity.csv"
dfEmployerCoverageRaceEthnicity = pd.read_csv(path,skiprows=2)
dfEmployerCoverageRaceEthnicity.drop(columns='Footnotes', inplace=True)
dfEmployerCoverageRaceEthnicity.dropna(thresh=20,inplace=True)
dfEmployerCoverageRaceEthnicity=fit_State_val_column(dfEmployerCoverageRaceEthnicity)
dfEmployerCoverageRaceEthnicity=concat_suicide_multy_year(dfEmployerCoverageRaceEthnicity, 7)
dfEmployerCoverageRaceEthnicity.drop(columns=["Total"],axis=1,inplace = True)
dfEmployerCoverageRaceEthnicity = filling_NaN_NSD(dfEmployerCoverageRaceEthnicity)

dfEmployerCoverageRaceEthnicity=marge_general_with_crimes(df_Y_S,dfEmployerCoverageRaceEthnicity)
dfEmployerCoverageRaceEthnicity.dropna(axis=0,thresh=10, inplace=True)
dfEmployerCoverageRaceEthnicity.rename(columns={"White":"P_D Emp_Cover: White",
                                                "Black":"P_D Emp_Cover: Black",
                                                "Hispanic":"P_D Emp_Cover: Hispanic",
                                                "Asian/Native Hawaiian and Pacific Islander":"P_D Emp_Cover: Asian/Hawaiian",
                                                "American Indian/Alaska Native":"P_D Emp_Cover: Indian/Alaska Nativ" ,
                                                "Multiple Races":"P_D Emp_Cover: Multiple Races"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfEmployerCoverageRaceEthnicity, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0,6115.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VIRGINIA,3021642.0,9942.0,260.0,1113.0,2227.0,6342.0,74059.0,7514.0,60704.0,5841.0,310.0,17130,0.005669,12.8,18.3,245.0,993.0,79.0,532.0,917.0,260.0,50.0,30.0,1266.0,1547.0,0.82,838.0,428.0,1266.0,5017100.0,15431

In [28]:
#Employment Status
path=direct+'Employment Status.csv'
dfEmploymentStatus = pd.read_csv(path,skiprows=2)
dfEmploymentStatus.drop(columns='Footnotes', inplace=True)
dfEmploymentStatus.dropna(thresh=20,inplace=True)
dfEmploymentStatus=fit_State_val_column(dfEmploymentStatus)
dfEmploymentStatus=concat_suicide_multy_year(dfEmploymentStatus, 4)
dfEmploymentStatus.drop(columns='Total', inplace=True)
dfEmploymentStatus=marge_general_with_crimes(df_Y_S,dfEmploymentStatus)
dfEmploymentStatus.dropna(axis=0,thresh=10, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfEmploymentStatus, how="outer")
df_Y_S_KFF


c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,1835500.0,197500.0,395400.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0,6115.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0,25179400.0,3121200.0,4036700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,2019,VI

In [29]:
#Health Insurance Coverage of the Total Population
path=direct+'Health Insurance Coverage of the Total Population.csv'
dfHealthInsurance = pd.read_csv(path,skiprows=2)
dfHealthInsurance.drop(columns='Footnotes', inplace=True)
dfHealthInsurance.dropna(thresh=20,inplace=True)
dfHealthInsurance=fit_State_val_column(dfHealthInsurance)
dfHealthInsurance=concat_suicide_multy_year(dfHealthInsurance, 7)
dfHealthInsurance.drop(columns='Total', inplace=True)
dfHealthInsurance=marge_general_with_crimes(df_Y_S,dfHealthInsurance)
dfHealthInsurance.dropna(axis=0,thresh=10, inplace=True)

dfHealthInsurance.rename(columns={"Employer":"Ins_Cov_P Employer",
                                  "Non-Group":"Ins_Cov_P Non-Group",
                                  "Medicaid":"Ins_Cov_P Medicaid",
                                  "Medicare":"Ins_Cov_P Medicare",
                                  "Military":"Ins_Cov_P Military",
                                  "Uninsured":"Ins_Cov_P  Uninsured",
                                  "Total":"Ins_Cov_P Total"} , inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfHealthInsurance, how="outer")
dfHealthInsurance
df_Y_S_KFF.shape

df_Y_S_KFF


c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,1835500.0,197500.0,395400.0,1169100.0,135300.0,587600.0,391900.0,54300.0,495800.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.0,Democrat,0,1,47239.0,102645.0,15773.0,10546.0,26441.0,6115.0,1525100.0,310100.0,4010200.0,713100.0,32500.0,121800.0,1525100.0,310100.0,4010200.0,

In [30]:
#Nonelderly Adult Poverty Rate by Sex
path=direct+'Nonelderly Adult Poverty Rate by Sex.csv'
dfAdultPovertyRatebySex = pd.read_csv(path,skiprows=2)
dfAdultPovertyRatebySex.drop(columns='Footnotes', inplace=True)
dfAdultPovertyRatebySex.dropna(thresh=20,inplace=True)
dfAdultPovertyRatebySex=fit_State_val_column(dfAdultPovertyRatebySex)
dfAdultPovertyRatebySex=concat_suicide_multy_year(dfAdultPovertyRatebySex, 3)
dfAdultPovertyRatebySex=marge_general_with_crimes(df_Y_S,dfAdultPovertyRatebySex)
dfAdultPovertyRatebySex.dropna(axis=0,thresh=10, inplace=True)

dfAdultPovertyRatebySex.rename(columns={"Male":"Poverty Male",
                                        "Female":"Poverty Female",
                                        "Total":"Poverty Total"}, inplace=True)

df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfAdultPovertyRatebySex, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,1835500.0,197500.0,395400.0,1169100.0,135300.0,587600.0,391900.0,54300.0,495800.0,123200.0,166900.0,290100.0
4,2010,CALIFORNIA,25481060.0,119015.0,1257.0,5966.0,46049.0,65743.0,721486.0,157556.0,450874.0,113056.0,5501.0,106430,0.004177,10.3,10.6,1063.0,205.0,456.0,326.0,1442.0,130.0,302.0,34.0,1929.0,4057.0,0.48,1225.0,704.0,1929.0,14600800.0,2042700.0,13825100.0,4777200.0,145000.0,125500.0,999300.0,96940.0,26540.0,123480.

In [31]:
#Population Distribution by Age
path=direct+'Population Distribution by Age.csv'
dfPopulationDistributionbyAge = pd.read_csv(path,skiprows=2)
dfPopulationDistributionbyAge.drop(columns='Footnotes', inplace=True)
dfPopulationDistributionbyAge.dropna(thresh=20,inplace=True)
dfPopulationDistributionbyAge=fit_State_val_column(dfPopulationDistributionbyAge)
dfPopulationDistributionbyAge=concat_suicide_multy_year(dfPopulationDistributionbyAge, 7)
dfPopulationDistributionbyAge.drop(columns='Total', inplace=True)
dfPopulationDistributionbyAge=marge_general_with_crimes(df_Y_S,dfPopulationDistributionbyAge)
dfPopulationDistributionbyAge.dropna(axis=0,thresh=10, inplace=True)

dfPopulationDistributionbyAge.rename(columns={"Children 0-18":"P_D Children 0-18",
                                   "Adults 19-25":"P_D Adults 19-25",
                                   "Adults 26-34":"P_D Adults 26-34",
                                   "Adults 35-54":"P_D Adults 35-54",
                                  "Adults 55-64":"P_D Adults 55-64",
                                  "65+":"P_D 65+",
                                  "Total":"P_D Total"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfPopulationDistributionbyAge, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,1835500.0,197500.0,395400.0,1169100.0,135300.0,587600.0,391900.0,54300.0,495800.0,123200.0,166900.0,290100.0,736800.0,258500.0,315400.0,766200.0,351300.0,405700.0
4,2010,

In [32]:
#Population Distribution by Citizenship Status
path=direct+'Population Distribution by Citizenship Status.csv'
dfPopulationDistributionbyCitizenshipStatus = pd.read_csv(path,skiprows=2)
dfPopulationDistributionbyCitizenshipStatus.drop(columns='Footnotes', inplace=True)
dfPopulationDistributionbyCitizenshipStatus.dropna(thresh=20,inplace=True)
dfPopulationDistributionbyCitizenshipStatus=fit_State_val_column(dfPopulationDistributionbyCitizenshipStatus)
dfPopulationDistributionbyCitizenshipStatus=concat_suicide_multy_year(dfPopulationDistributionbyCitizenshipStatus, 3)
dfPopulationDistributionbyCitizenshipStatus.drop(columns='Total', inplace=True)


dfPopulationDistributionbyCitizenshipStatus=marge_general_with_crimes(df_Y_S,dfPopulationDistributionbyCitizenshipStatus)
dfPopulationDistributionbyCitizenshipStatus.dropna(axis=0,thresh=10, inplace=True)
#dfPopulationDistributionbyCitizenshipStatus.rename(columns={"Total":"P_D Total Citizen"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfPopulationDistributionbyCitizenshipStatus, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0,5720700.0,536200.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,1835500.0,197500.0,395400.0,1169100.0,135300.0,587600.0,391900.0,54300.0,495800.0,123200.0,1669

In [33]:
#Population Distribution by Family Structure
path=direct+'Population Distribution by Family Structure.csv'
dfPopulationDistributionbyFamilyStructure = pd.read_csv(path,skiprows=2)
dfPopulationDistributionbyFamilyStructure.drop(columns='Footnotes', inplace=True)
dfPopulationDistributionbyFamilyStructure.dropna(thresh=20,inplace=True)
dfPopulationDistributionbyFamilyStructure=fit_State_val_column(dfPopulationDistributionbyFamilyStructure)
dfPopulationDistributionbyFamilyStructure=concat_suicide_multy_year(dfPopulationDistributionbyFamilyStructure, 3)
dfPopulationDistributionbyFamilyStructure.drop(columns=["Total"],axis=1,inplace = True)
dfPopulationDistributionbyFamilyStructure=marge_general_with_crimes(df_Y_S,dfPopulationDistributionbyFamilyStructure)
dfPopulationDistributionbyFamilyStructure.dropna(axis=0,thresh=10, inplace=True)

dfPopulationDistributionbyFamilyStructure.rename(columns={"Adults with Children":"P_D Adults with Childre",
                                   "Adults with No Children":"P_D Adults with No Children",
                                   "Total":"P_D Total Family Structure"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfPopulationDistributionbyFamilyStructure, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0,5720700.0,536200.0,1327000.0,2362800.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,0,4112.0,11303.0,2038.0,2729.0,4769.0,534.0,319800.0,88800.0,65700.0,8400.0,4400.0,8200.0,319800.0,88800.0,65700.0,8400.0,

In [34]:
#Poverty Rate by Age
# changed from precent to numeric#

path=direct+'Poverty Rate by Age.csv'
dfPovertyRatebyAge = pd.read_csv(path,skiprows=2)
dfPovertyRatebyAge.drop(columns='Footnotes', inplace=True)
dfPovertyRatebyAge.dropna(thresh=20,inplace=True)
dfPovertyRatebyAge=fit_State_val_column(dfPovertyRatebyAge)
dfPovertyRatebyAge=concat_suicide_multy_year(dfPovertyRatebyAge, 4)

# edit and filling nan values
dfPovertyRatebyAge.drop(columns=["Total"],axis=1,inplace = True)
dfPovertyRatebyAge=filling_NaN_NSD(dfPovertyRatebyAge)

## merge 
dfPovertyRatebyAge=marge_general_with_crimes(df_Y_S,dfPovertyRatebyAge)
dfPovertyRatebyAge.dropna(axis=0,thresh=10, inplace=True)
dfPovertyRatebyAge.rename(columns={"Children 0-18A":"Poverty Children 0-18",
                                   "Adults 19-64":"Poverty Adults 19-64",
                                   "65+":"Poverty 65+"}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfPovertyRatebyAge, how="outer")
dfPovertyRatebyAge
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0,5720700.0,536200.0,1327000.0,2362800.0,415300.0,612400.0,69300.0
3,2010,ARKANSAS,1654464.0,11049.0,95.0,901.0,2179.0,7874.0,76304.0,22290.0,51769.0,3918.0,370.0,9208,0.005566,15.5,12.5,118.0,57.0,56.0,12.0,177.0,10.0,11.0,11.0,189.0,350.0,0.54,101.0,88.0,189.0,2113800.0,443000.0,178300.0,32300.0,17600.0,4700.0,44300.0,2120.0,642.0,2762.0,Republican,1,

In [35]:
#Poverty Rate by Race Ethnicity
path=direct+'Poverty Rate by RaceEthnicity.csv'
dfPovertyRatebyRace = pd.read_csv(path,skiprows=2)
dfPovertyRatebyRace.drop(columns='Footnotes', inplace=True)
dfPovertyRatebyRace.dropna(thresh=20,inplace=True)

dfPovertyRatebyRace=fit_State_val_column(dfPovertyRatebyRace)
dfPovertyRatebyRace=concat_suicide_multy_year(dfPovertyRatebyRace, 7)

# edit and filling nan values
dfPovertyRatebyRace.drop(columns=["Total"],axis=1,inplace = True)
dfPovertyRatebyRace= filling_NaN_NSD(dfPovertyRatebyRace)


## merge
dfPovertyRatebyRace=marge_general_with_crimes(df_Y_S,dfPovertyRatebyRace)
dfPovertyRatebyRace.dropna(axis=0,thresh=10, inplace=True)

dfPovertyRatebyRace.rename(columns={"White":"Poverty White",
                      "Black":"Poverty Black",
                      "Hispanic":"Poverty Hispanic",
                      "Asian/Native Hawaiian and Pacific Islander":"Poverty Asian/Hawaiian ",
                      "American Indian/Alaska Native":"Poverty Indian/Alaska Native",
                      "Multiple Races":"Poverty Multiple Races"},inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfPovertyRatebyRace, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0,5720700.0,536200.0,1327000.0,2362800.0,415300.0,612400.0,69300.0,407300.0,55000.0,502700.0,20500.0,95600.0,15800.0
3,2010,ARKANSAS,1654464.0

In [36]:
#Distribution of the Total Population by Federal Poverty Level (above and below 100% FPL) 

path=direct+'Distribution of the Total Population by Federal Poverty Level (above and below 100% FPL).csv'
fd_FPL_100 = pd.read_csv(path,skiprows=2)
fd_FPL_100.drop(columns='Footnotes', inplace=True)
fd_FPL_100.dropna(thresh=20,inplace=True)

fd_FPL_100=fit_State_val_column(fd_FPL_100)
fd_FPL_100=concat_suicide_multy_year(fd_FPL_100, 2)

# edit and filling nan values
fd_FPL_100= filling_NaN_NSD(fd_FPL_100)

## merge
fd_FPL_100=marge_general_with_crimes(df_Y_S,fd_FPL_100)
fd_FPL_100.dropna(axis=0,thresh=10, inplace=True)

fd_FPL_100.rename(columns={"At or Below 99%":"Poverty: At or Below 99%", "100%+":"Poverty: 100%+"},inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, fd_FPL_100, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races,Poverty: At or Below 99%,Poverty: 100%+
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0,888400.0,3763100.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0,75800.0,608800.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,875500.0,5720700.0,536200.0,1327000.0,2362800.0,415300.0,612400.0,69300.0

In [37]:
#Distribution of the Total Population by Federal Poverty Level (above and below 200% FPL) 
path=direct+'Distribution of the Total Population by Federal Poverty Level (above and below 200% FPL).csv'
fd_FPL_200 = pd.read_csv(path,skiprows=2)
fd_FPL_200.drop(columns='Footnotes', inplace=True)
fd_FPL_200.dropna(thresh=20,inplace=True)

fd_FPL_200=fit_State_val_column(fd_FPL_200)
fd_FPL_200=concat_suicide_multy_year(fd_FPL_200, 2)

# edit and filling nan values
fd_FPL_200= filling_NaN_NSD(fd_FPL_200)
## merge
fd_FPL_200=marge_general_with_crimes(df_Y_S,fd_FPL_200)
fd_FPL_200.dropna(axis=0,thresh=10, inplace=True)

fd_FPL_200.rename(columns={"Under 200%":"Poverty: Under 200%", "200%+":"Poverty: 200%+"},inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, fd_FPL_200, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races,Poverty: At or Below 99%,Poverty: 100%+,Poverty: Under 200%,Poverty: 200%+
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0,888400.0,3763100.0,1871000.0,2780400.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0,75800.0,608800.0,184700.0,499900.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,277200.0,335200.0,612400.0,1691700.0,582700.0,745100.0,1634700.0,727300.0,

In [38]:
#Distribution of the Total Population by Federal Poverty Level (above and below 400% FPL) 
path=direct+'Distribution of the Total Population by Federal Poverty Level (above and below 400% FPL).csv'
fd_FPL_400 = pd.read_csv(path,skiprows=2)
fd_FPL_400.dropna(thresh=2,inplace=True)

fd_FPL_400=fit_State_val_column(fd_FPL_400)
fd_FPL_400=concat_suicide_multy_year(fd_FPL_400, 2)

# # edit and filling nan values
fd_FPL_400= filling_NaN_NSD(fd_FPL_400)

## merge
fd_FPL_400=marge_general_with_crimes(df_Y_S,fd_FPL_400)
fd_FPL_400.dropna(axis=0,thresh=10, inplace=True)

fd_FPL_400.rename(columns={"Under 400%":"Poverty: Under 400%", "400%+":"Poverty: 400%+"},inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, fd_FPL_400, how="outer")
df_Y_S_KFF

c:\users\אלמוג\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races,Poverty: At or Below 99%,Poverty: 100%+,Poverty: Under 200%,Poverty: 200%+,Poverty: Under 400%,Poverty: 400%+
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0,888400.0,3763100.0,1871000.0,2780400.0,3284500.0,1366900.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0,75800.0,608800.0,184700.0,499900.0,384300.0,300300.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0,115600.0,1061900.0,2

In [39]:
#Total Gross State Product (GSP) (millions of current dollars)
path=direct+'Total Gross State Product (GSP) (millions of current dollars).csv'
dfGSP = pd.read_csv(path,skiprows=2)
dfGSP.dropna(inplace=True)

dfGSP=edding_year_missing_values(dfGSP,2010,0,'min-std')
dfGSP=fit_State_val_column(dfGSP)
dfGSP=concat_suicide(dfGSP)
dfGSP=marge_general_with_crimes(df_Y_S,dfGSP)
dfGSP.dropna(thresh=10, inplace=True)
dfGSP.rename(columns={"Total Gross State Product":"GSP",}, inplace=True)
df_Y_S_KFF = pd.merge(df_Y_S_KFF, dfGSP, how="outer")
df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races,Poverty: At or Below 99%,Poverty: 100%+,Poverty: Under 200%,Poverty: 200%+,Poverty: Under 400%,Poverty: 400%+,GSP
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0,888400.0,3763100.0,1871000.0,2780400.0,3284500.0,1366900.0,165503.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0,75800.0,608800.0,184700.0,499900.0,384300.0,300300.0,46476.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0

In [ ]:
search_NSD(df_Y_S_KFF) # search for Nan And NSD

In [ ]:
df_Y_S_KFF.info()
#df_Y_S_KFF.to_excel('./proj_datasets/df_Y_S_KFF.xlsx',engine='xlsxwriter')

In [44]:
df_Y_S_KFF

,Year,State,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1,Total_Police force,Police Force Population Ratio,Suicide Rate P100K,OD D P100K,"Natural and Semisynthetic Opioids (e.g. oxycodone, hydrocodone)","Synthetic Opioids, other than Methadone (e.g. fentanyl, tramadol)",Methadone,Heroin,Opi ODD White N-H,Opi ODD Black N-H,Opi ODD Hispanic,Opi ODD other,Opioid Overdose Deaths,All Drug Overdose Deaths,Opioid Overdose Deaths as a Percent of All Drug Overdose Deaths,Opi ODD Male,Opi ODD Female,Opi ODD Total Gender,P_D: White,P_D: Black,P_D: Hispanic,P_D: Asian,P_D: Indian/Alaska Nativ,P_D: Hawaiian/Pacific Islander Nativ,P_D: Multiple Races,Individuals Homeless,People in Families with Children Homeless,Total People Experiencing Homelessness,Governor Political Affiliation,Republican,Democrat,Chlamydia Man,Chlamydia Women,Gonorrhea Man,Gonorrhea Women,Gonorrhea Cases,Syphilis Cases,P_D_Uninsured: White,P_D_Uninsured: Black,P_D_Uninsured: Hispanic,P_D_Uninsured: Asian/Hawaiian,P_D_Uninsured: Indian/Alaska Nativ,P_D_Uninsured: Multiple Races,P_D Emp_Cover: White,P_D Emp_Cover: Black,P_D Emp_Cover: Hispanic,P_D Emp_Cover: Asian/Hawaiian,P_D Emp_Cover: Indian/Alaska Nativ,P_D Emp_Cover: Multiple Races,At Least 1 Full Time Worker,Part Time Workers,Non Workers,Ins_Cov_P Employer,Ins_Cov_P Non-Group,Ins_Cov_P Medicaid,Ins_Cov_P Medicare,Ins_Cov_P Military,Ins_Cov_P Uninsured,Poverty Male,Poverty Female,Poverty Total,P_D Children 0-18,P_D Adults 19-25,P_D Adults 26-34,P_D Adults 35-54,P_D Adults 55-64,P_D 65+,Citizen,Non-Citizen,P_D Adults with Childre,P_D Adults with No Children,Children 0-18,Poverty Adults 19-64,Poverty 65+,Poverty White,Poverty Black,Poverty Hispanic,Poverty Asian/Hawaiian,Poverty Indian/Alaska Native,Poverty Multiple Races,Poverty: At or Below 99%,Poverty: 100%+,Poverty: Under 200%,Poverty: 200%+,Poverty: Under 400%,Poverty: 400%+,GSP
0,2010,ALABAMA,2056773.0,7661.0,116.0,606.0,2294.0,4904.0,81245.0,19180.0,62921.0,4891.0,121.0,13296,0.006464,14.0,11.8,82.0,34.0,78.0,12.0,177.0,10.0,11.0,11.0,187.0,547.0,0.34,117.0,70.0,187.0,3133000.0,1212100.0,176400.0,49300.0,23000.0,3100.0,57200.0,4553.0,1493.0,6046.0,Republican,1,0,6877.0,20030.0,3430.0,4432.0,7933.0,781.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,366300.0,208900.0,86800.0,10400.0,3600.0,8700.0,2954000.0,327000.0,731000.0,2195700.0,232900.0,842600.0,599200.0,94100.0,686800.0,201400.0,289800.0,491200.0,1182300.0,436900.0,523500.0,1282300.0,587100.0,639400.0,4531400.0,120000.0,965800.0,1864000.0,330100.0,491200.0,67100.0,414000.0,395400.0,53400.0,4700.0,4800.0,16200.0,888400.0,3763100.0,1871000.0,2780400.0,3284500.0,1366900.0,165503.0
1,2010,ALASKA,470248.0,3450.0,16.0,465.0,552.0,2425.0,16471.0,2014.0,13183.0,1274.0,129.0,2398,0.005099,22.9,11.6,38.0,10.0,25.0,12.0,49.0,10.0,11.0,11.0,62.0,83.0,0.75,32.0,30.0,62.0,439300.0,20500.0,37900.0,35500.0,98500.0,5200.0,47700.0,1118.0,745.0,1863.0,Republican,1,0,2058.0,3960.0,575.0,698.0,1273.0,15.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,58400.0,2500.0,8300.0,9500.0,32900.0,10900.0,502300.0,45600.0,84100.0,349500.0,19000.0,108700.0,39600.0,43700.0,124100.0,18600.0,25300.0,43900.0,196800.0,64000.0,85700.0,200000.0,85500.0,52700.0,661800.0,22800.0,159600.0,275600.0,28500.0,43900.0,3400.0,36200.0,75400.0,2700.0,5800.0,23800.0,6600.0,75800.0,608800.0,184700.0,499900.0,384300.0,300300.0,46476.0
2,2010,ARIZONA,4856365.0,19925.0,254.0,1296.0,6494.0,11881.0,157989.0,39563.0,109293.0,17568.0,912.0,24426,0.005030,17.0,17.5,327.0,68.0,106.0,90.0,483.0,19.0,91.0,11.0,626.0,1098.0,0.57,372.0,254.0,626.0,3628600.0,236200.0,1857400.0,165900.0,252000.0,11500.0,105200.0,9226.0,4485.0,13711.0,Republican,1,0,7331.0,19529.0,1696.0,1553.0,3249.0,905.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,397200.0,33400.0,506900.0,21000.0,77500.0,16200.0,4137200.0,476200.0,768000.0,2707800.0,318300.0,1273700.0,779600.0